In [1]:
from data_loading import *
from torch.utils.data import Dataset, DataLoader

# One method to rule them all

In [2]:
train_dataset, test_dataset = get_dataset('gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr', asTensors=False)

/Users/amarmesic/miniconda3/envs/mlg/lib/python3.10/site-packages/xarray/core/dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


Model surface dataset size 75.83941642194986 GiB
Lag features created successfully
Train-test split done
Scaling done
Converted to tensors


In [3]:
torch.save(train_dataset, 'datasets/train_dataset.pt')
torch.save(test_dataset, 'datasets/test_dataset.pt')

In [4]:
train_dataset = torch.load('datasets/train_dataset.pt')
train_dataset.X.shape

torch.Size([233, 64, 32, 12])

In [ ]:
# Parameters
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train model...

## If you want to try out with custom arguments...

In [10]:
era5 = get_xarray_dataset('gs://weatherbench2/datasets/era5/1959-2022-6h-128x64_equiangular_with_poles_conservative.zarr')
era5 = era5[[
    '10m_u_component_of_wind', '10m_v_component_of_wind', 
    '10m_wind_speed', 
    '2m_temperature', 
    'mean_sea_level_pressure',
    'surface_pressure', 'toa_incident_solar_radiation_6hr', 
    'total_cloud_cover', 
    'total_column_water_vapour',
    'total_precipitation_6hr']]
era5 = era5.sel(time=slice('2008-01-01', '2010-12-31'))
era5 = era5.to_array()

Model surface dataset size 303.3556067124009 GiB


/Users/amarmesic/miniconda3/envs/mlg/lib/python3.10/site-packages/xarray/core/dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [11]:
era5

<xarray.DataArray (variable: 10, time: 4384, longitude: 128, latitude: 64)>
dask.array<stack, shape=(10, 4384, 128, 64), dtype=float32, chunksize=(1, 48, 128, 64), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -90.0 -87.14 -84.29 ... 84.29 87.14 90.0
  * longitude  (longitude) float64 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
  * time       (time) datetime64[ns] 2008-01-01 ... 2010-12-31T18:00:00
  * variable   (variable) object '10m_u_component_of_wind' ... 'total_precipi...

In [12]:
X_train, y_train, X_test, y_test = get_weather_dataset_from_array(era5, obs_window=2, pred_window=1, overlap=False, group_by_time=True, test_size=0.2, return_tensors=True)
X_train, y_train, X_test, y_test

Lag features created successfully
Train-test split done


/Users/amarmesic/miniconda3/envs/mlg/lib/python3.10/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing with an out-of-order index is generating 13 times more chunks
  return self.array[key]


Scaling done
Converted to tensors


(tensor([[[[ 1.0411e+00, -8.0397e-01, -1.1119e-01,  ...,  8.2780e-01,
             1.9364e+00,  1.8966e+00],
           [ 2.4998e-01, -8.5273e-01,  3.4695e-01,  ...,  9.7969e-01,
             1.5041e+00,  4.4692e-01],
           [-1.3584e+00,  3.0742e-01,  1.2458e+00,  ...,  7.6568e-01,
             3.6333e-01, -3.6505e-01],
           ...,
           [-9.1410e-02, -2.1146e+00,  1.9340e+00,  ...,  5.1682e-01,
            -8.0925e-01, -1.8848e-01],
           [-4.8557e-01, -2.1475e+00,  1.9013e+00,  ..., -3.1046e+00,
            -8.5587e-01, -5.0912e-01],
           [-1.8981e-01, -1.6189e+00,  9.0907e-01,  ..., -6.2648e+00,
            -8.3600e-01, -5.4549e-01]],
 
          [[ 1.0572e+00, -7.3434e-01, -1.2708e-01,  ...,  8.3211e-01,
             1.9168e+00,  1.8701e+00],
           [ 3.1386e-01, -7.9823e-01,  2.9780e-01,  ...,  9.9031e-01,
             1.4610e+00,  5.3347e-01],
           [-1.3332e+00,  2.2534e-01,  1.2088e+00,  ...,  4.3429e-01,
             3.2155e-01, -3.5764e-01],


In [13]:
torch.save(X_train, 'datasets/128x64_10f_3y_2obs/X_train.pt')
torch.save(y_train, 'datasets/128x64_10f_3y_2obs/y_train.pt')
torch.save(X_test, 'datasets/128x64_10f_3y_2obs/X_test.pt')
torch.save(y_test, 'datasets/128x64_10f_3y_2obs/y_test.pt')

In [ ]:
X_train = torch.load('datasets/X_train.pt')

In [11]:
class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset instances
train_dataset = WeatherDataset(X_train, y_train)
test_dataset = WeatherDataset(X_test, y_test)

NameError: name 'X_train' is not defined

In [12]:
# Parameters
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train model...

Now we train the model